# RVCGP 
## Simple two node function

Test, wheter RVCGP is able to find exact solution, to simple two node function with prescription:
$$
f(x) = x_0*x_1 + x_0
$$

Generate train data:

In [1]:
import numpy as np

def target_function(X):
    return X[:,0] * X[:,1] + X[:,0]

_X = np.linspace(-5, 5, 20)
x, y = np.meshgrid(_X, _X)
X_train = np.dstack((x, y)).reshape(400, 2)
X_train = np.c_[X_train, np.ones(len(X_train))]
y_train = target_function(X_train)

Add tengp into the mix.

In [2]:
import tengp

def pdivide(x, y):
    return np.divide(x, y, out=np.copy(x), where=x!=0)

funset = tengp.FunctionSet()
funset.add(np.add, 2)
funset.add(np.subtract, 2)
funset.add(np.multiply, 2)
funset.add(pdivide, 2)

rv_params = tengp.Parameters(3, 1, 1, 2, funset, real_valued=True)
params = tengp.Parameters(3, 1, 1, 2, funset)

bounds = tengp.individual.IndividualBuilder(params).create().bounds[:]

Baseline using classic representation and point mutation.

Add PyGMO optimizer

In [3]:
import pygmo as pg
from sklearn.metrics import mean_squared_error

class cost_function:
    def __init__(self, X, Y, params, bounds):
        self.params = params
        self.bounds = bounds
        self.X = X
        self.Y = Y
    
    def fitness(self, x):      
        individual = tengp.individual.NPIndividual(
            list(x), self.bounds, self.params
        )

        pred = individual.transform(self.X).reshape(400)
                
        try:
            return [mean_squared_error(pred, self.Y)]
        except ValueError:
            return [10000000000]
        
    def get_bounds(self):
        lower = [0]*len(self.bounds)
        #lower[-1] = 4
        return (lower, [b for b in self.bounds])

# Hyperparameter tuning with bayesian optimization (BBOptimizer)

In [4]:
import bboptimizer

In [5]:
space_conf = [
    {'name': 'omega', 'domain': (0.1, 1), 'type': 'continuous'},
    {'name': 'max_vel', 'domain': (0.01, 1), 'type': 'continuous'},
    {'name': 'eta1', 'domain': (0.01, 4), 'type': 'continuous'},
    {'name': 'eta2', 'domain': (0.01, 4), 'type': 'continuous'}
]

In [9]:
def target(params):
    omega = params['omega']
    max_vel = params['max_vel']
    eta1 = params['eta1']
    eta2 = params['eta2']
    
    problem = pg.problem(cost_function(X_train, y_train, rv_params, bounds))
    algorithm = pg.algorithm(pg.pso(
        gen=50,
        omega=omega, 
        max_vel=max_vel,
        eta1=eta1,
        eta2=eta2
    ))
    algorithm.set_verbosity(1)
    pop = pg.population(problem, 100)
    pop = algorithm.evolve(pop)
    return pop.champion_f[0]

opt = bboptimizer.Optimizer(target, space_conf, sampler='bayes', backend='gpy')

In [10]:
res = opt.search(num_iter=100)

100%|██████████| 100/100 [07:32<00:00,  4.52s/it]


In [8]:
best_parameters = opt.results[0][-1]
best_performance = opt.results[1][-1]
print(f'Fitness: {best_performance}, parameters: {best_parameters}')

Fitness: 5.6899246761089256e-05, parameters: {'omega': 0.3943166227644823, 'max_vel': 1.0, 'eta1': 4.0, 'eta2': 4.0}


In [11]:
best_parameters = opt.results[0][-1]
best_performance = opt.results[1][-1]
print(f'Fitness: {best_performance}, parameters: {best_parameters}')

Fitness: 2.974879602552487e-05, parameters: {'omega': 0.41729814172560553, 'max_vel': 1.0, 'eta1': 0.44581217810358853, 'eta2': 4.0}


In [13]:
opt.results

([{'eta1': 1.5286405031285046,
   'eta2': 3.2061578524742522,
   'max_vel': 0.15208932734995126,
   'omega': 0.7135951420469816},
  {'eta1': 3.5265173975003563,
   'eta2': 1.1158259658630947,
   'max_vel': 0.24984068100561924,
   'omega': 0.5948422815374363},
  {'eta1': 3.5265173975003563,
   'eta2': 1.1158259658630947,
   'max_vel': 0.24984068100561924,
   'omega': 0.5948422815374363},
  {'eta1': 3.5265173975003563,
   'eta2': 1.1158259658630947,
   'max_vel': 0.24984068100561924,
   'omega': 0.5948422815374363},
  {'eta1': 3.33644637570932,
   'eta2': 1.3047383301840416,
   'max_vel': 0.33130219372042347,
   'omega': 0.5555837710655448},
  {'eta1': 3.33644637570932,
   'eta2': 1.3047383301840416,
   'max_vel': 0.33130219372042347,
   'omega': 0.5555837710655448},
  {'eta1': 1.9805043456554807,
   'eta2': 2.5113092880923498,
   'max_vel': 0.36500149532885695,
   'omega': 0.5844037289029839},
  {'eta1': 1.9805043456554807,
   'eta2': 2.5113092880923498,
   'max_vel': 0.3650014953288569

In [ ]:
print(np.unique([x['variant'] for x in opt.results[0]]))
print(np.unique([x['neighb_type'] for x in opt.results[0]]))

# Hyperparameter tuning using Bayesian optimization (BayesianOptimization)

In [ ]:
from bayes_opt import BayesianOptimization

def bo_target(max_vel, omega, variant, neighb_type):
    
    variant = round(variant)
    neighb_type = round(neighb_type)
    
    problem = pg.problem(cost_function(X_train, y_train, rv_params, bounds))
    algorithm = pg.algorithm(pg.pso(
        gen=250,
        omega=omega, 
        max_vel=max_vel,
        seed=42
    ))
    algorithm.set_verbosity(1)
    pop = pg.population(problem, 20)
    pop = algorithm.evolve(pop)
    return -pop.champion_f[0]

bo = BayesianOptimization(f=bo_target,
                          pbounds={
                            'max_vel': (0.01, 1),
                            'omega': (0, 1),
                            'variant': (1, 6),
                            'neighb_type': (1, 4)
                          },
                          verbose=1)

bo.maximize(init_points=2, n_iter=100, acq="ucb", kappa=10)

In [ ]:
bo.res['max']